#### This script reads in zstat images from group-level feat results and outputs tables listing all significant clusters and the relevant statistics. You can choose to output a cluster table that lists either 1) all clusters found by feat, or 2) all clusters that overlap with ROI masks.

In [ ]:
import glob
import subprocess
from os.path import exists
import pandas as pd
import numpy as np

#group_stats_path = '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/group_analyses/group_stats_3May22_N23_motionRegressors_Z3.1_p0.001_FLAME1+2_noTempDerivs_modelv3.gfeat/'
group_stats_path = '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/group_analyses/group_stats_3May22_N23_motionRegressors_Z3.1_p0.0001_FLAME1+2_noTempDerivs_modelv3.gfeat/'
all_zstat_dir = glob.glob(group_stats_path + 'cope*.feat')
threshold = 0.0001
filter_clusters = False # print out only e.g. top 10 largest clusters?

Function toolbox

In [ ]:
def format_cluster_output(output, zstat_num, stat_type):
    # output = raw output of cluster function;
    # zstat_num = number corresponding to cope image
    # zstat_type = "activation" or "deactivation", depending on type of statistical image
    t = [x.split('\t') for x in output.decode('utf-8').split('\n')]
    t = pd.DataFrame(t[1:-1], columns=t[0]) # omit first and last lines; these are col names & blank line respectively
    t['zstat'] = zstat_num
    t['stat_type'] = stat_type
    return t

def add_atlas_info(df, atlas_name):
    # df = a pandas dataframe that has been formatted by format_cluster_output()
    # atlas_name = an fsl atlas (str)
    df[atlas_name] = np.nan
    for c in range(len(df)):
        coords = ''.join([df.loc[c,"MAX X (mm)"]+','+df.loc[c,"MAX Y (mm)"]+','+df.loc[c,"MAX Z (mm)"]])
        output = subprocess.check_output(['atlasquery','-a',atlas_name,'-c',coords]).decode('utf-8')
        output = output.strip('\n').split('<br>')[-1]
        df.loc[c,atlas_name] = output
    return df

def get_fslstats_output(image, roi):
    # image = path to a thresholded stats image
    # roi = path to an roi mask
    # Outputs: volume of overlap between image & roi, plus coordinates & value of max voxel
    range_output = subprocess.check_output(['fslstats',image,'-k',roi,'-R'])
    vol_output = subprocess.check_output(['fslstats',image,'-k',roi,'-V']) # get volume of nonzero voxels

    coord_vox = subprocess.check_output(['fslstats',image,'-k',roi,'-x']) # get coordinates of maximum voxel
    coord_vox = [int(x) for x in coord_vox.decode('utf-8').strip(' \n').split(' ')] # format
    ps = subprocess.run(['echo',str(coord_vox[0]),str(coord_vox[1]),str(coord_vox[2])], check=True, capture_output=True)
    coord_mm = subprocess.run(['img2stdcoord','-img',image,'-std',image,'-vox'], # convert to MNI coordinates
                      input=ps.stdout, capture_output=True)
    coord_mm = [int(x) for x in coord_mm.stdout.decode('utf-8').strip('\n').split(' ') if x !='']
    
    return range_output, vol_output, coord_mm

def format_fslstats_output(range_output, vol_output, coord_mm, cluster_ind, zstat_num, stat_type, r):
    # range_output, vol_output, coord_mm = outputs of fslstats function;
    # zstat_num = number corresponding to cope image
    # zstat_type = "activation" or "deactivation", depending on type of statistical image
    # r = str representing roi name
    t = pd.DataFrame(columns=['zstat','stat_type','roi','min_z','max_z','n_voxels','x_mm','y_mm','z_mm','Cluster_index'])
    # Get range
    range_output = range_output.decode('utf-8').strip('\n').split(' ')
    maxval = float(range_output[1])
    minval = float(range_output[0])
    if (maxval==0) & (minval==0):
        maxval = 'N/A'
        minval = 'N/A'
    
    if maxval != 'N/A':
        # Get volume
        vol_output = float(vol_output.decode('utf-8').strip('\n').split(' ')[0])

        # Save to dataframe
        t.loc[0,'zstat'] = zstat_num
        t.loc[0,'stat_type'] = stat_type
        t.loc[0,'roi'] = r
        t.loc[0,'min_z'] = minval
        t.loc[0,'max_z'] = maxval
        t.loc[0,'n_voxels'] = vol_output # 'N/A' if maxval=='N/A' else vol_output
        t.loc[0,'x_mm'] = coord_mm[0] # 'N/A' if maxval=='N/A' else coord_mm[0]
        t.loc[0,'y_mm'] = coord_mm[1] # 'N/A' if maxval=='N/A' else coord_mm[1]
        t.loc[0,'z_mm'] = coord_mm[2] # 'N/A' if maxval=='N/A' else coord_mm[2]
        t.loc[0,'Cluster_index'] = cluster_ind
    return t

## Method 1. Get stats from all clusters in each zstat image
This will output a table that shows each cluster, the zstat image it came from, the max Z stat from that cluster, x/y/z coordinates from the max voxel, and whether it's an activation or deactivation

In [ ]:
all_cluster_stats = pd.DataFrame(columns=['Cluster Index', 'Voxels', 'MAX', 'MAX X (mm)', 'MAX Y (mm)',
       'MAX Z (mm)', 'COG X (mm)', 'COG Y (mm)', 'COG Z (mm)', 'zstat',
       'stat_type']) # Use this format if you want to output MNI coordinates instead of voxel location
# all_cluster_stats = pd.DataFrame(columns=['Cluster Index', 'Voxels', 'MAX', 'MAX X (vox)', 'MAX Y (vox)',
#        'MAX Z (vox)', 'COG X (vox)', 'COG Y (vox)', 'COG Z (vox)', 'zstat',
#        'stat_type'])
for path in all_zstat_dir:
    zstat_num = int(path.split('/')[-1].strip('.feat').strip('cope'))
    # print("Running cluster stats for zstat ", zstat_num)

    im1 = path + '/thresh_zstat1.nii.gz' # '/stats/zstat1.nii.gz'
    im2 = path + '/thresh_zstat2.nii.gz' # '/stats/zstat2.nii.gz'
    
    if exists(im1) and exists(im2):
        output = subprocess.check_output(['cluster','-i',im1,'-t',str(threshold),'--mm']) # --mm tag outputs MNI coordinates
        t1 = format_cluster_output(output, zstat_num, 'activation')

        output = subprocess.check_output(['cluster','-i',im2,'-t',str(threshold),'--mm'])
        t2 = format_cluster_output(output, zstat_num, 'deactivation')
        
        all_cluster_stats = pd.concat([all_cluster_stats, t1], ignore_index=True)
        all_cluster_stats = pd.concat([all_cluster_stats, t2], ignore_index=True)

Optional: Constrain to only the top 10 biggest clusters from each zstat image:

In [ ]:
if filter_clusters:
    top_thresh = 10
    all_cluster_stats['Cluster Index'] = [int(x) for x in all_cluster_stats['Cluster Index']]

    # Sort out activations
    top_activations = pd.DataFrame(columns=all_cluster_stats.columns)
    for z in all_cluster_stats.zstat.unique():
        temp = all_cluster_stats.loc[(all_cluster_stats.zstat==z) & (all_cluster_stats.stat_type=='activation'),:]
        temp = temp.reset_index()
        if len(temp) > 0:
            max_cluster_val = max(temp['Cluster Index'])
            top_cluster_inds = [x for x in range(len(temp)) if temp['Cluster Index'][x] > (max_cluster_val-top_thresh)]
            top_activations = pd.concat([top_activations,temp.loc[top_cluster_inds,:]], ignore_index=True)
    top_activations = top_activations.sort_values(by=['zstat','Cluster Index','stat_type'])
    top_activations = top_activations.reset_index()

    top_deactivations = pd.DataFrame(columns=all_cluster_stats.columns)
    for z in all_cluster_stats.zstat.unique():
        temp = all_cluster_stats.loc[(all_cluster_stats.zstat==z) & (all_cluster_stats.stat_type=='deactivation'),:]
        temp = temp.reset_index()
        if len(temp) > 0:
            max_cluster_val = max(temp['Cluster Index'])
            top_cluster_inds = [x for x in range(len(temp)) if temp['Cluster Index'][x] > (max_cluster_val-top_thresh)]
            top_deactivations = pd.concat([top_deactivations,temp.loc[top_cluster_inds,:]], ignore_index=True)
    top_deactivations = top_deactivations.sort_values(by=['zstat','Cluster Index','stat_type'])
    top_deactivations = top_deactivations.reset_index()
else:
    top_activations = all_cluster_stats.loc[all_cluster_stats.stat_type=='activation',:]
    top_activations = top_activations.sort_values(by=['zstat','Cluster Index','stat_type'])
    
    top_deactivations = all_cluster_stats.loc[all_cluster_stats.stat_type=='deactivation',:]
    top_deactivations = top_deactivations.sort_values(by=['zstat','Cluster Index','stat_type'])

Optional: Add atlas information for each cluster (subcortical & cortical). (This takes a while)

In [ ]:
# atlas1 = 'Harvard-Oxford Subcortical Structural Atlas'
# atlas2 = 'Harvard-Oxford Cortical Structural Atlas'

# top_activations = add_atlas_info(top_activations, atlas1)
# top_activations = add_atlas_info(top_activations, atlas2)
# top_deactivations = add_atlas_info(top_deactivations, atlas1)
# top_deactivations = add_atlas_info(top_deactivations, atlas2)

Save to file

In [ ]:
# Save info about top clusters to group feat path
top_activations.to_csv(group_stats_path+'/activation-ROI-stats-thresh'+str(threshold)+'.csv')
top_deactivations.to_csv(group_stats_path+'/deactivation-ROI-stats-thresh'+str(threshold)+'.csv')

## Method 2. Overlay ROIs over statistical image and get stats from within ROI
This will output a table that lists, for each ROI and each zstat image, whether there is any overlap between significant clusters in that zstat image and the ROI mask; if so, it will list in the table the number of voxels overlapping, the value and coordinates of the maximum voxel in the overlapping area, and the index of the cluster.

First create a copy of each zstat image with clusters indexed. This will allow us to determine, for each ROI, the index of the cluster that overlaps with it:

In [ ]:
for path in all_zstat_dir:
    im1 = path+'/thresh_zstat1.nii.gz'
    im2 = path+'/thresh_zstat2.nii.gz'
    
    if exists(im1) and exists(im2):
        subprocess.check_output(['cluster','-i',im1,'-t',str(threshold),'-o',path+'/thresh_zstat1_indexed.nii.gz','--no_table'])
        subprocess.check_output(['cluster','-i',im2,'-t',str(threshold),'-o',path+'/thresh_zstat2_indexed.nii.gz','--no_table'])

Next search through each zstat image and determine whether any clusters overlap with each ROI. NOTE: your ROI masks must be binarized for this to work.

In [ ]:
all_cluster_stats = pd.DataFrame(columns=['zstat','stat_type','roi','min_z','max_z','n_voxels','x_mm','y_mm','z_mm','Cluster_index'])
ROIs = ['ACC_cluster_cope7','Ce_amygdala','DRN','HB','insular_cortex_cope7','NAc','PAG','pulvinar','s_colliculus','VTA_SN']
roi_path = '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/group_analyses/masks/selected_masks_for_ROI_analysis/1.5mm/'

for path in all_zstat_dir:
    zstat_num = int(path.split('/')[-1].strip('.feat').strip('cope'))
    # print("Running cluster stats for zstat ", zstat_num)

    im1 = path + '/thresh_zstat1.nii.gz' # '/stats/zstat1.nii.gz'
    im2 = path + '/thresh_zstat2.nii.gz' # '/stats/zstat2.nii.gz'
    
    if exists(im1) and exists(im2):
        for r in ROIs: # check if each thresholded image overlaps with this ROI
            # Image 1 (activations)
            range_output, vol_output, coord_mm = get_fslstats_output(im1, roi_path+r+'_bin.nii.gz') # Make sure you select the binarized mask version
            # Get cluster index
            if float(vol_output.decode('utf-8').strip('\n').split(' ')[0])!=0: # if ROI overlaps, get index of cluster
                im_ind = path + '/thresh_zstat1_indexed.nii.gz'
                cluster_ind = subprocess.check_output(['fslstats',im_ind,'-k',roi_path+r+'_bin.nii.gz','-M'])
                cluster_ind = int(float(cluster_ind.decode('utf-8').strip(' \n')))
            else:
                cluster_ind = 'N/A'
            t1 = format_fslstats_output(range_output, vol_output, coord_mm, cluster_ind, zstat_num, 'activation', r)
            all_cluster_stats = pd.concat([all_cluster_stats, t1], ignore_index=True)

            # Image 2 (deactivations)
            range_output, vol_output, coord_mm = get_fslstats_output(im2, roi_path+r+'_bin.nii.gz')
            # Get cluster index
            if float(vol_output.decode('utf-8').strip('\n').split(' ')[0])!=0: # if ROI overlaps, get index of cluster
                im_ind = path + '/thresh_zstat2_indexed.nii.gz'
                cluster_ind = subprocess.check_output(['fslstats',im_ind,'-k',roi_path+r+'_bin.nii.gz','-M'])
                cluster_ind = int(float(cluster_ind.decode('utf-8').strip(' \n')))
            else:
                cluster_ind = 'N/A'
            t2 = format_fslstats_output(range_output, vol_output, coord_mm, cluster_ind, zstat_num, 'deactivation', r)
            all_cluster_stats = pd.concat([all_cluster_stats, t2], ignore_index=True)

In [ ]:
all_cluster_stats = all_cluster_stats.sort_values(by=['zstat','stat_type','Cluster_index'])
all_cluster_stats

Save to file


In [ ]:
sort_activations = False
if sort_activations:
    top_activations = all_cluster_stats.loc[all_cluster_stats.stat_type=='activation',:]
    top_activations = top_activations.sort_values(by=['zstat'])

    top_deactivations = all_cluster_stats.loc[all_cluster_stats.stat_type=='deactivation',:]
    top_deactivations = top_deactivations.sort_values(by=['zstat'])
    
    top_activations.to_csv(group_stats_path+'/activation-ROI-stats-thresh'+str(threshold)+'.csv')
    top_deactivations.to_csv(group_stats_path+'/deactivation-ROI-stats-thresh'+str(threshold)+'.csv')
else:
    all_cluster_stats.to_csv(group_stats_path+'/all-ROI-stats-indexed-thresh'+str(threshold)+'.csv')